In [59]:
import pandas as pd # used for DataFrames

In [60]:
# Function to organize .xyz file data into python list with correct data types of features
# Lists will be used as input data in Pandas DataFrame
def xyz_to_list(xyz_file):
    data_list = []
    for row in xyz_file:
        # splits rows of .xyz data to list of strings
        row_array = row.split()
        # changes data type of values in list to correct data type
        for i in range(len(row_array)):
            if row_array[i].find(".") != -1:
                row_array[i] = float(row_array[i])
            elif row_array[i].isdigit() or row_array[i][1:].isdigit():
                row_array[i] = int(row_array[i])
        # adds list of row data to the main list
        data_list.append(row_array)
    # returns organized list of data
    return data_list

In [61]:
# Magnetic .xyz file to python list
clovis_mag = "CLOVIS/clovis_mag.xyz" # magnetic data file path
xyz_mag = open(clovis_mag, "r") # create xyz file object
mag_list = xyz_to_list(xyz_mag) # magnetic data formatted in python list

print(mag_list[0])

[16, 211, 115600, 103, 1976, 34.696, -101.9878, 139.9, 53303.7, -176.9, -9999.9, 'TO', 107.0]


In [62]:
# DataFrame for magnetic data
mag_DataFrame = pd.DataFrame(mag_list, 
            columns=["line", "fid", "time", "day", "year", "latitude", "longitude", "radalt", "totmag", "resmag", "diurnal", "geology", "resmagCM4"])

# First 10 rows of mag data
mag_DataFrame.head(10)

,line,fid,time,day,year,latitude,longitude,radalt,totmag,resmag,diurnal,geology,resmagCM4
0,16,211,115600,103,1976,34.6960,-101.9878,139.9,53303.7,-176.9,-9999.9,TO,107.0
1,16,212,115600,103,1976,34.6960,-101.9878,139.9,53303.7,-176.9,-9999.9,TO,107.0
2,16,213,115600,103,1976,34.6968,-101.9883,137.8,53304.4,-176.8,-9999.9,TO,107.2
3,16,214,115600,103,1976,34.6974,-101.9888,136.2,53306.5,-175.0,-9999.9,TO,109.0
4,16,215,115600,103,1976,34.6981,-101.9893,135.3,53304.8,-177.1,-9999.9,TO,106.8
5,16,216,115600,103,1976,34.6987,-101.9898,135.0,53305.2,-177.0,-9999.9,TO,107.0
6,16,217,115600,103,1976,34.6992,-101.9903,134.4,53306.6,-175.9,-9999.9,TO,108.1
7,16,218,115600,103,1976,34.6997,-101.9908,136.9,53304.8,-177.8,-9999.9,TO,106.2
8,16,219,115600,103,1976,34.7004,-101.9913,138.7,53306.7,-176.2,-9999.9,TO,107.7
9,16,220,115600,103,1976,34.7012,-101.9918,138.7,53306.1,-177.3,-9999.9,TO,106.8


In [63]:
# Radiometric xyz file to python lists
clovis_rad = "CLOVIS/clovis_rad.xyz" # radiometric data file path
xyz_rad = open(clovis_rad, "r") # create xyz file object
rad_list = xyz_to_list(xyz_rad) # radiometric data formatted in python list

print(rad_list[0])

[1, 1, 171900, 102, 1976, 34.0081, -104.0169, 141.1, -266.0, '-99', '-9999', 140.0, 9.0, 27.0, -999.9, -999.9, -999.9, 3614.0, 601.0, 20.8, 651.3]
